In [1]:
import local_models.local_models
import local_models.algorithms
import local_models.utils
import local_models.linear_projections
import local_models.loggin
import local_models.TLS_models
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.cluster
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import sklearn.datasets
import sklearn.decomposition
import logging
import ml_battery.log
import time
import os
import pcl
import pcl.pcl_visualization
import mayavi
import mayavi.mlab

import subprocess
#on headless systems, tmux: "Xvfb :1 -screen 0 1280x1024x24 -auth localhost", then "export DISPLAY=:1" in the jupyter tmux
mayavi.mlab.options.offscreen = True



logger = logging.getLogger(__name__)

#reload(local_models.local_models)
#reload(lm)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')


In [2]:
kernel_names = {
    local_models.local_models.GaussianKernel: 'gaussian',
    local_models.local_models.TriCubeKernel: 'tricube'
}

In [3]:
mpl.rcParams['figure.figsize'] = [8.0, 8.0]

In [4]:
bun = pcl.load("../data/bun0.pcd")
bunny = np.asarray(bun)

In [5]:
KERNEL=local_models.local_models.GaussianKernel

project_dir = "../data/pcl/bun0_{}".format(kernel_names[KERNEL])
os.makedirs(project_dir, exist_ok=1)

In [6]:
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(room)

In [7]:
linear_models = local_models.local_models.LocalModels(local_models.TLS_models.LinearODR_mD(2))
linear_models.fit(bunny)

In [24]:
bandwidth = 3*np.average(linear_models.index.query(bunny, k=2)[0][:,1])

In [25]:
for converged in local_models.algorithms.local_tls_shift_till_convergence(linear_models, bunny, tol=1e-8,
                                     kernel=KERNEL(bandwidth),
                                     report=False):
    pass

In [26]:
converged

array([[ 0.00487494,  0.11126457,  0.03734089],
       [-0.00191818,  0.11182683,  0.03764273],
       [-0.01050489,  0.11141071,  0.03788622],
       ...,
       [-0.06519622,  0.17987472, -0.05381454],
       [-0.06984834,  0.17880122, -0.05245961],
       [-0.07785027,  0.17445776, -0.04472706]], dtype=float32)

In [27]:
cloud = pcl.PointCloud()
cloud.from_array(converged)
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(cloud)

In [28]:
range_pct = 0.2
grid_steps = 20
data_mins, data_maxes, data_ranges = local_models.linear_projections.min_max_range(bunny)
graph_bounds = local_models.linear_projections.sane_graph_bounds(data_mins, data_maxes, data_ranges, range_pct)

grid_limits = tuple(map(lambda i: slice(graph_bounds[0,i], graph_bounds[1,i], grid_steps*1j), range(graph_bounds.shape[1])))
grid = np.mgrid[grid_limits]
gridder = local_models.utils.Grid2Vec()
grid = gridder.fit_transform(grid)

In [29]:
for converged2 in local_models.algorithms.local_tls_shift_till_convergence(linear_models, grid, tol=1e-10,
                                     kernel=KERNEL(bandwidth),
                                     report=False):
    pass

In [30]:
converged2

array([[-0.06859611,  0.03610813,  0.00019016],
       [-0.06947882,  0.03422496, -0.0007538 ],
       [-0.0703089 ,  0.03285433, -0.00226235],
       ...,
       [ 0.02695358,  0.1280962 ,  0.0112135 ],
       [ 0.02647546,  0.12739984,  0.01289547],
       [ 0.02610835,  0.12693077,  0.01398126]])

In [31]:
cloud = pcl.PointCloud()
cloud.from_array(converged2.astype('float32'))
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(cloud)

In [32]:
mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(800, 800))
#mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
nodes = mayavi.mlab.points3d(converged2[:,0], converged2[:,1], converged2[:,2], scale_mode='scalar', color=(1,0,0))
nodes.mlab_source.dataset.point_data.scalars = np.ones(converged2.shape[0])*0.1
bunodes = mayavi.mlab.points3d(bunny[:,0], bunny[:,1], bunny[:,2], scale_mode='scalar', color=(0,1,0))
bunodes.mlab_source.dataset.point_data.scalars = np.ones(bunny.shape[0])*0.1


#mayavi.mlab.axes()

#mayavi.mlab.view(views[0][1],views[0][0])
data_avg = np.average(converged2, axis=1)
mayavi.mlab.view(azimuth=0, elevation=30, distance=1, focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
title = "converged_g00020"
mayavi.mlab.savefig(os.path.join(project_dir, "{}.png".format(title)))
mayavi.mlab.clf()